# Add trial labels

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/social-neuro-modeling'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

In [17]:
from utilities import helper

## Functions

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.read_pickle("./proc/rce_pilot_2_05_lfp_spectral_sleap_spikes_labels.pkl")

OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs



In [9]:
VIDEO_TO_FRAME_AND_SUBJECT_DF

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,subject,...,start_stop_timestamps,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps,video_file,trial_time_stamps,trial_label,trial_competition_closeness
0,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120...","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[lose, win, win, lose, win, lose, lose, lose, ...","[non_comp, non_comp, non_comp, non_comp, non_c..."
1,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[win, lose, lose, win, lose, win, win, win, lo...","[non_comp, non_comp, non_comp, non_comp, non_c..."


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

# Documentation

- cohort
- session_dir
- original_file
- tone_frames
- box_1_port_entry_frames
- box_2_port_entry_frames
- video_name
- session_path
- recording
- subject
- all_subjects
- first_timestamp
- last_timestamp
- video_timestamps
    - box_top_left
- box_top_right
- reward_port
- box_bottom_left
- box_bottom_right
- agent
- subject_locations
- agent_locations
- file_path
- tracked_subject
- in_video_subjects
- notes
- sleap_filename
- sleap_fileroot
- start_stop_frame
- start_stop_timestamps

- tone_timestamps
- box_1_port_entry_timestamps
- box_2_port_entry_timestamps

- lfp_timestamps
    - mPFC_lfp_trace
    - MD_lfp_trace
    - LH_lfp_trace
    - BLA_lfp_trace
    - vHPC_lfp_trace
- power_timestamps
    - mPFC_power_all_frequencies_all_windows
    - MD_power_all_frequencies_all_windows
    - LH_power_all_frequencies_all_windows
    - BLA_power_all_frequencies_all_windows
    - vHPC_power_all_frequencies_all_windows
    - power_calculation_frequencies
- coherence_timestamps
    - BLA_LH_coherence_all_frequencies_all_windows
    - BLA_MD_coherence_all_frequencies_all_windows
    - BLA_mPFC_coherence_all_frequencies_all_windows
    - BLA_vHPC_coherence_all_frequencies_all_windows
    - LH_MD_coherence_all_frequencies_all_windows
    - LH_mPFC_coherence_all_frequencies_all_windows
    - LH_vHPC_coherence_all_frequencies_all_windows
    - MD_mPFC_coherence_all_frequencies_all_windows
    - MD_vHPC_coherence_all_frequencies_all_windows
    - mPFC_vHPC_coherence_all_frequencies_all_windows
    - coherence_calculation_frequencies
- granger_timestamps
    - BLA_LH_granger_all_frequencies_all_windows
    - LH_BLA_granger_all_frequencies_all_windows
    - BLA_MD_granger_all_frequencies_all_windows
    - MD_BLA_granger_all_frequencies_all_windows
    - BLA_mPFC_granger_all_frequencies_all_windows
    - mPFC_BLA_granger_all_frequencies_all_windows
    - BLA_vHPC_granger_all_frequencies_all_windows
    - vHPC_BLA_granger_all_frequencies_all_windows
    - LH_MD_granger_all_frequencies_all_windows
    - MD_LH_granger_all_frequencies_all_windows
    - LH_mPFC_granger_all_frequencies_all_windows
    - mPFC_LH_granger_all_frequencies_all_windows
    - LH_vHPC_granger_all_frequencies_all_windows
    - vHPC_LH_granger_all_frequencies_all_windows
    - MD_mPFC_granger_all_frequencies_all_windows
    - mPFC_MD_granger_all_frequencies_all_windows
    - MD_vHPC_granger_all_frequencies_all_windows
    - vHPC_MD_granger_all_frequencies_all_windows
    - mPFC_vHPC_granger_all_frequencies_all_windows
    - vHPC_mPFC_granger_all_frequencies_all_windows
    - granger_calculation_frequencies

- recording_name
- spike_clusters
- spike_times
- neuron_average_timestamps
    - neuron_average_fr
- video_file
- trial_time_stamps
    - trial_label
    - trial_competition_closeness

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [10]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,subject,...,start_stop_timestamps,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps,video_file,trial_time_stamps,trial_label,trial_competition_closeness
0,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120...","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[lose, win, win, lose, win, lose, lose, lose, ...","[non_comp, non_comp, non_comp, non_comp, non_c..."
1,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[win, lose, lose, win, lose, win, win, win, lo...","[non_comp, non_comp, non_comp, non_comp, non_c..."


In [14]:
for col in VIDEO_TO_FRAME_AND_SUBJECT_DF.columns:
    print("`{}`:".format(col))

`cohort`:
`session_dir`:
`original_file`:
`tone_frames`:
`box_1_port_entry_frames`:
`box_2_port_entry_frames`:
`video_name`:
`session_path`:
`recording`:
`subject`:
`all_subjects`:
`first_timestamp`:
`last_timestamp`:
`video_timestamps`:
`tone_timestamps`:
`box_1_port_entry_timestamps`:
`box_2_port_entry_timestamps`:
`lfp_timestamps`:
`mPFC_lfp_trace`:
`MD_lfp_trace`:
`LH_lfp_trace`:
`BLA_lfp_trace`:
`vHPC_lfp_trace`:
`mPFC_power_all_frequencies_all_windows`:
`MD_power_all_frequencies_all_windows`:
`LH_power_all_frequencies_all_windows`:
`BLA_power_all_frequencies_all_windows`:
`vHPC_power_all_frequencies_all_windows`:
`power_timestamps`:
`power_calculation_frequencies`:
`BLA_LH_coherence_all_frequencies_all_windows`:
`BLA_MD_coherence_all_frequencies_all_windows`:
`BLA_mPFC_coherence_all_frequencies_all_windows`:
`BLA_vHPC_coherence_all_frequencies_all_windows`:
`LH_MD_coherence_all_frequencies_all_windows`:
`LH_mPFC_coherence_all_frequencies_all_windows`:
`LH_vHPC_coherence_all_frequ

In [19]:
example_tone_timestamp_range = VIDEO_TO_FRAME_AND_SUBJECT_DF["tone_timestamps"].iloc[0][0]

In [20]:
example_tone_timestamp_range

array([1099250, 1299250], dtype=int32)

In [24]:
example_timestamps = VIDEO_TO_FRAME_AND_SUBJECT_DF["video_timestamps"].iloc[0]

In [31]:
example_timestamps.shape

(68495,)

In [25]:
example_timestamps

array([      -2,     1384,     2770, ..., 68590840, 68592226, 68593612],
      dtype=int32)

In [29]:
example_poses = VIDEO_TO_FRAME_AND_SUBJECT_DF["subject_locations"].iloc[0]

In [30]:
example_poses.shape

(68495, 6, 2)

In [34]:
filtered_timestamps, filtered_items = helper.filter_by_timestamp_range(start=example_tone_timestamp_range[0], 
                                 stop=example_tone_timestamp_range[1],
                                timestamps=example_timestamps,
                                items=example_poses)

In [35]:
filtered_timestamps

array([1100378, 1101764, 1101764, 1103150, 1104536, 1105922, 1105922,
       1107308, 1108694, 1108694, 1110079, 1111465, 1112851, 1112851,
       1114237, 1115623, 1116387, 1117009, 1118395, 1119780, 1120733,
       1121166, 1122552, 1123938, 1123938, 1125324, 1126710, 1128096,
       1128096, 1129482, 1130867, 1130867, 1132253, 1133639, 1133639,
       1135025, 1136411, 1137797, 1137797, 1139183, 1140569, 1141477,
       1141954, 1143340, 1144726, 1146112, 1146112, 1147498, 1148884,
       1148884, 1150270, 1151655, 1153041, 1153041, 1154427, 1155813,
       1155813, 1157199, 1158585, 1159971, 1159971, 1161357, 1162742,
       1162742, 1164128, 1165514, 1166900, 1166900, 1168286, 1169672,
       1171058, 1171058, 1172444, 1173829, 1175215, 1175215, 1176601,
       1177987, 1177987, 1179373, 1180759, 1182145, 1182145, 1183530,
       1184916, 1184916, 1186302, 1187688, 1187688, 1189074, 1190460,
       1191846, 1192326, 1193232, 1194617, 1196003, 1196071, 1197389,
       1198775, 1200

In [36]:
filtered_items

array([[[52.83974324, 10.03478168],
        [55.63376198, 10.75414115],
        [40.25656128, 26.33319168],
        [50.77391002, 15.3571852 ],
        [48.14345461, 16.21076701],
        [54.97021435,  9.96474175]],

       [[52.83940773, 10.04831234],
        [55.68484512, 10.78680199],
        [40.82821373, 25.59292134],
        [55.58025624, 10.72666922],
        [53.37350259, 11.88241165],
        [55.08428404,  9.96375601]],

       [[52.83907221, 10.061843  ],
        [55.73592825, 10.81946283],
        [41.39986618, 24.85265101],
        [55.57263471, 10.3599931 ],
        [53.4308987 , 11.65536706],
        [55.19835372,  9.96277027]],

       ...,

       [[49.0356057 , 30.62167685],
        [46.28717836, 29.25648251],
        [46.77093545, 31.41282742],
        [50.64554161, 23.62344011],
        [50.15950349, 26.98621861],
        [47.37058033, 30.6799566 ]],

       [[48.48907588, 30.62111829],
        [45.76774378, 29.21766508],
        [45.73793902, 31.38854113],
       